# Code for solving CSP map coloring problem

# Assignment Description
In this problem, you will work with a Jupyter Notebook script that models a map coloring
problem as a Constraint Satisfaction Problem (CSP). In this task, you will implement based
on the provided starter code to understand how Minimum Remaining Values (MRV) and
Least Constraining Value (LCV) heuristics work.

# Variables and Domains

Variables: The states of Australia (WA, NT, SA, Q, NSW, V, T).    
Domains: A set of colors{ {Red,Blue,  Gree }.

# Restrictions and Heuristics
## a) No Adjacent Color: 
    No two neighboring states can have the same color. This constraint is fundamental to the map coloring problem and ensures that the solution is valid. The adjacency of states is determined by the provided matrix, where a value of ‘1’ at position (i, j) indicates that states i and j are neighbors.
## b) Minimum-Remaining-Values (MRV) Heuristic for Choosing States: 
    This heuristic chooses the variable (i.e., the state) with the fewest remaining legal colors to be assigned.
## c) Least Constraining Value (LCV) Heuristic for Choosing Colors: 
    The LCV heuristic chooses the color that rules out the fewest number of color choices for neighboring states.k.

In [1]:
# import libraries
import sys
import copy

In [2]:
# class representation of map to color
class Map():
    def __init__(self, states, colors, matrix):
        self.states = states # list of string states
        self.colors = colors # list of char colors
        self.matrix = matrix # adjacency matrix
        self.n = len(states) # number of states [nodes]
        self.m = len(colors) # number of colors [domain]

    # returns number of constraints on a states given index
    def findConstr(self, solution, states):
        constraints = 0
        for i in self.matrix[states]:
            # only consider unassigned states
            if i == '1' and solution[states] == -1:
                constraints += 1
        return constraints
    
    # determines if color is valid assignment
    def isLegal(self, solution, states, color):
        for i in range(self.n):
            if(self.matrix[states][i] == '1' and color == solution[i]):
                return False
        return True

In [3]:
# functions to update variables not in Map class
# returns updated version of variable
# parameters
    # domDict: dictionary of all states mapped to valid domain colors
    # sol: dictionary of states mapped to their solution colors, -1 if not colored
    # history: list of expanded states
    # states: index of states to expand
    # color: index of color to assign

def updateDom(domDict, states, color):
    update = domDict[states]
    update[color] = ''  # Use color index to update domain
    return update

def updateSolDict(sol, states, color):
    update = sol
    update[states] = color
    return update

def updHist(history, states, color):
    update = copy.deepcopy(history)
    update.append(states)
    return update

In [4]:
# functions to implement forward checking
def addInference(mc, solution, domDict, states, colorIndex):
    # remove color from domain of all neighboring states
    for i in range(mc.n):
        if(mc.matrix[states][i] == '1' and solution[i] == -1):
            domDict[states] = updateDom(domDict, states, colorIndex)
    # check for failure
    for dom in domDict[states]:
        if dom != '':
            return (True, domDict)
    return (False, domDict)

def removeInference(mc, domDict, solution, states, color):
    for i in range(mc.n):
        if(mc.matrix[states][i] == '1' and solution[i] == -1):
            domDict[i][color] = mc.colors[color]
    return domDict

In [5]:
# function to implement mrv heuristic
# returns index of states to evaluate
# parameters
    # mc: Map instance
    # solution: dictionary of  states mapped to their solution colors, -1 if not colored
    # domDict: dictionary of all states mapped to valid domain colors
def calcMRV(mc, solution, domDict):
    '''
    # Initialize a variable to store the index of the state with the minimum remaining values
    # ...

    # Initialize a variable to store the minimum number of remaining values found so far
    # ...

    # Loop through all states
    for i in range(mc.n):
        # Check if the state is unassigned (i.e., its color is -1 in the solution dictionary)
        # ...

            # Count the number of available colors for this state
            # Hint: Iterate through the domain dictionary (domDict) for this state
            #       and count the colors that are still available (not empty)
            # ...

            # Compare this count with the minimum number of remaining values
            # If it's smaller, update the state index and the minimum count
            # ...

    # Return the index of the state with the minimum remaining values
    # If no unassigned state is left, return -1
    # ...
    '''
    unassigned_states = [i for i in range(mc.n) if solution[i] == -1]
    mrv_state = None
    min_remaining_values = float('inf')

    for state in unassigned_states:
        remaining_values = len([color for color in domDict[state] if color != ''])
        if remaining_values < min_remaining_values:
            min_remaining_values = remaining_values
            mrv_state = state

    return mrv_state if mrv_state is not None else -1

# Function to calculate the least constraining value (color)
# Returns colors sorted based on how many times they appear in the domains of neighboring states
# Parameters
    # mc: Map instance
    # domDict: dictionary of all states mapped to valid domain colors
    # solution: dictionary of states mapped to their solution colors, -1 if not colored
    # states: index of states to expand
def calcLCV(mc, domDict, solution, states):
    '''
    # Initialize a dictionary to count the occurrence of each color in the neighboring states
    # ...

    # Loop through all states
    for i in range(mc.n):
        # Check if the state is a neighbor and is unassigned
        # Hint: Use the adjacency matrix (mc.matrix) and the solution dictionary
        # ...

            # For each available color in the domain of the neighboring state,
            # increment its count in the color count dictionary
            # ...

    # Sort and return the colors based on their occurrence count
    # Hint: The color with the least occurrences should come first
    #       as it is the least constraining
    # ...
    '''
    color_choices = domDict.get(states, [])
    lcv_scores = []

    for color in color_choices:
        lcv_score = 0
        for neighbor_state_index in range(mc.n):
            if mc.matrix[states][neighbor_state_index] == '1' and solution[neighbor_state_index] == -1:
                if color in domDict[neighbor_state_index]:
                    lcv_score += 1
        lcv_scores.append(lcv_score)

    sorted_colors = [color for _, color in sorted(zip(lcv_scores, color_choices))]
    return sorted_colors

In [6]:
# recursive backtracking function to execute map coloring
# returns when all states have been colored
# parameters
    # mc: Map instance
    # domDict: dictionary of all states mapped to valid domain colors
    # solution: dictionary of  states mapped to their solution colors, -1 if not colored
    # history: list of expanded states
    # states: index of states to expand
def coloring(mc, domDict, solution, history, states):
    if states == -1:
        return True  # All states are assigned
    
    for color in calcLCV(mc, domDict, solution, states):  # Use LCV to choose color
        colorIndex = mc.colors.index(color)
        if mc.isLegal(solution, states, colorIndex):
            # remove color from domain of updated states
            domDict[states] = updateDom(domDict, states, colorIndex)

            # FORWARD CHECKING
            fwc = addInference(mc, solution, domDict, states, colorIndex)
            if(fwc[0]):
                domDict = fwc[1]
                # update solution list
                solution = updateSolDict(solution, states, colorIndex)
            else:
                domDict = removeInference(mc, solution, domDict, states, colorIndex)

            # update history list
            history = updHist(history, states, colorIndex)

            if(len(history) <= mc.n):
                coloring(mc, domDict, solution, history, calcMRV(mc, solution, domDict))
            else:
                return True
    return False

# function to execute map coloring
def runMapColoring(states, colors, adjMatrix):
    # create Map
    mc = Map(states, colors, adjMatrix)

    # collections for search
    domDict = {i: copy.deepcopy(colors) for i in range(len(states))}
    solution = {i: -1 for i in range(len(states))}
    history = []

    # call backtracking function
    failure = coloring(mc, domDict, solution, history, calcMRV(mc, solution, domDict))

    if failure:
        return failure
    else:
        # print out results
        toPrint(mc, solution)
        
# output to file
# parameters
    # solution: list with selected colors
def toPrint(mc, solution):   
    print("Solution found:")
    for c in solution:
        print(mc.states[c] + " = " + mc.colors[solution[c]])
    print('T = Any color (Since Tasmania is not adjacent to any other state)')

In [7]:
# function to read information
def main():
    values = '6 3\n' # variable and domain number
    states = 'NSW NT Q SA WA V\n'
    colors = 'R G B\n'
    adj = ['0 0 1 1 0 1\n', '0 0 1 1 1 0\n', '1 1 0 1 0 0\n', '1 1 1 0 1 1\n', '0 1 0 1 0 0 \n', '1 0 0 1 0 0']

    #represent n and m
    n = values.split()[0]
    m = values.split()[1]

    #represent states
    states = states.split()

    #represent colors
    colors = colors.split()

    #represent adjacency matrix
    adjMatrix = []
    tempArr = []
    for row in adj:
        for char in row:
            if char == '1' or char == '0':
                tempArr.append(char)
        adjMatrix.append(tempArr)
        tempArr = []

    # run backtracking algorithm
    runMapColoring(states, colors, adjMatrix)

#run script
main()

Solution found:
NSW = B
NT = B
Q = G
SA = R
WA = G
V = G
T = Any color (Since Tasmania is not adjacent to any other state)
